In [1]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from dataset import get_transforms

with initialize(version_base=None, config_path="config", job_name="test_app"):
    cfg = compose(config_name="root", overrides=['data=task3'])
    
transform = get_transforms('train', cfg)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'


In [3]:
from glob import glob
from monai.data import Dataset
from monai.data import DataLoader
datapaths = []

for image_path in glob('/workspace/dataset/Task03_Liver/imagesTr/*'):
    datapaths.append({'image':image_path, 'label':image_path.replace('imagesTr', 'labelsTr')})
    
ds = Dataset(datapaths, transform)
dl = DataLoader(ds, num_workers=8, batch_size=2)

In [4]:
for batch in dl:
    print(batch['image'].shape, batch['label'].shape)

torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128]) torch.Size([2,